# GeoMasterPy Data Analysis Example

This notebook demonstrates geospatial analysis capabilities of GeoMasterPy.

In [ ]:
import ee
import geomasterpy as gmp
import pandas as pd

# Initialize Earth Engine
ee.Initialize()

## Load Sentinel-2 Data

In [ ]:
# Define area of interest (a region in California)
aoi = ee.Geometry.Rectangle([-121.5, 37.0, -121.0, 37.5])

# Load Sentinel-2 data
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2023-06-01', '2023-08-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median() \
    .clip(aoi)

print(f"Loaded Sentinel-2 image with bands: {s2.bandNames().getInfo()}")

## Calculate Spectral Indices

In [ ]:
# Calculate multiple indices
indices_image = gmp.analysis.calculate_indices(s2, ['NDVI', 'NDWI', 'NDBI', 'EVI'])

print(f"Image with indices: {indices_image.bandNames().getInfo()}")

## Visualize Results

In [ ]:
# Create map
Map = gmp.Map(center=(37.25, -121.25), zoom=10)

# Add true color image
rgb_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.add_ee_layer(s2, rgb_vis, 'Sentinel-2 RGB')

# Add NDVI
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.add_ee_layer(indices_image.select('NDVI'), ndvi_vis, 'NDVI')

# Add colorbar
gmp.add_colorbar(Map, ndvi_vis, 'NDVI')

Map

## Image Statistics

In [ ]:
# Calculate statistics for the indices
stats = gmp.analysis.image_stats(
    indices_image, 
    aoi, 
    scale=20, 
    bands=['NDVI', 'NDWI', 'NDBI', 'EVI']
)

# Display as DataFrame
stats_df = pd.DataFrame(stats).T
print("Index Statistics:")
stats_df

## Unsupervised Classification

In [ ]:
# Perform k-means clustering
bands_for_classification = ['B2', 'B3', 'B4', 'B8', 'NDVI', 'NDWI']
classified = gmp.analysis.unsupervised_classification(
    indices_image,
    num_classes=5,
    scale=20,
    region=aoi,
    bands=bands_for_classification
)

# Visualization for classification
class_vis = {
    'min': 0,
    'max': 4,
    'palette': ['red', 'blue', 'green', 'yellow', 'purple']
}

Map.add_ee_layer(classified, class_vis, 'Land Cover Classes')

# Add legend
class_legend = {
    'Class 1': 'red',
    'Class 2': 'blue', 
    'Class 3': 'green',
    'Class 4': 'yellow',
    'Class 5': 'purple'
}
gmp.add_legend(Map, 'Land Cover Classes', class_legend)

## Time Series Analysis

In [ ]:
# Load time series data
time_series = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi.centroid()) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI').copyProperties(img, ['system:time_start']))

# Create time series chart
chart = gmp.viz.create_time_series_chart(
    time_series,
    aoi.centroid().buffer(1000),
    band='NDVI',
    reducer='mean',
    scale=20
)

chart

## Create Histogram

In [ ]:
# Create histogram for NDVI values
histogram = gmp.viz.create_histogram(
    indices_image,
    aoi,
    'NDVI',
    scale=20
)

histogram

## Export Data

In [ ]:
# Export NDVI image to Google Drive
export_task = gmp.export.export_image_to_drive(
    indices_image.select('NDVI'),
    description='NDVI_analysis',
    folder='GeoMasterPy_exports',
    region=aoi,
    scale=20
)

print(f"Export task started: {export_task.id if export_task else 'Failed'}")